## 1. Установка PySpark

In [1]:
!pip install pyspark

## 2. Импорт необходимых библиотек и создание проекта

In [2]:
import pyspark
from pyspark import SparkContext, SparkConf

In [3]:
sc = SparkContext(conf=SparkConf().setAppName("L1").setMaster("local[*]"))

In [4]:
sc

<SparkContext master=local[*] appName=L1>

## 3. Считываем данные

In [5]:
data_trip = sc.textFile("/content/data/trips.csv")
data_station = sc.textFile("/content/data/stations.csv")

## 4. Предобработка данных

In [6]:
trip_header=data_trip.first()
station_header=data_station.first()

In [7]:
trips=data_trip.filter(lambda x: x != trip_header).map(lambda x: x.split(","))
stations=data_station.filter(lambda x: x != station_header).map(lambda x: x.split(","))

In [8]:
trips.take(3)

[['4576',
  '63',
  '',
  'South Van Ness at Market',
  '66',
  '8/29/2013 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127'],
 ['4607',
  '',
  '8/29/2013 14:42',
  'San Jose City Hall',
  '10',
  '8/29/2013 14:43',
  'San Jose City Hall',
  '10',
  '661',
  'Subscriber',
  '95138'],
 ['4130',
  '71',
  '8/29/2013 10:16',
  'Mountain View City Hall',
  '27',
  '8/29/2013 10:17',
  'Mountain View City Hall',
  '27',
  '48',
  'Subscriber',
  '97214']]

In [9]:
stations.take(3)

[['2',
  'San Jose Diridon Caltrain Station',
  '37.329732',
  '-121.90178200000001',
  '27',
  'San Jose',
  '8/6/2013'],
 ['3',
  'San Jose Civic Center',
  '37.330698',
  '-121.888979',
  '15',
  'San Jose',
  '8/5/2013'],
 ['4',
  'Santa Clara at Almaden',
  '37.333988',
  '-121.894902',
  '11',
  'San Jose',
  '8/6/2013']]

In [10]:
# Индексирование по нулевому элементу
station_idx = stations.keyBy(lambda x: int(x[0]))

In [11]:
station_idx.take(3)

[(2,
  ['2',
   'San Jose Diridon Caltrain Station',
   '37.329732',
   '-121.90178200000001',
   '27',
   'San Jose',
   '8/6/2013']),
 (3,
  ['3',
   'San Jose Civic Center',
   '37.330698',
   '-121.888979',
   '15',
   'San Jose',
   '8/5/2013']),
 (4,
  ['4',
   'Santa Clara at Almaden',
   '37.333988',
   '-121.894902',
   '11',
   'San Jose',
   '8/6/2013'])]

In [12]:
# Индексирование по начальной и конечной станции
trips_start = trips.keyBy(lambda x: x[3])
trips_end = trips.keyBy(lambda x: x[6])

In [13]:
trips_start.take(3), trips_end.take(3)

([('South Van Ness at Market',
   ['4576',
    '63',
    '',
    'South Van Ness at Market',
    '66',
    '8/29/2013 14:14',
    'South Van Ness at Market',
    '66',
    '520',
    'Subscriber',
    '94127']),
  ('San Jose City Hall',
   ['4607',
    '',
    '8/29/2013 14:42',
    'San Jose City Hall',
    '10',
    '8/29/2013 14:43',
    'San Jose City Hall',
    '10',
    '661',
    'Subscriber',
    '95138']),
  ('Mountain View City Hall',
   ['4130',
    '71',
    '8/29/2013 10:16',
    'Mountain View City Hall',
    '27',
    '8/29/2013 10:17',
    'Mountain View City Hall',
    '27',
    '48',
    'Subscriber',
    '97214'])],
 [('South Van Ness at Market',
   ['4576',
    '63',
    '',
    'South Van Ness at Market',
    '66',
    '8/29/2013 14:14',
    'South Van Ness at Market',
    '66',
    '520',
    'Subscriber',
    '94127']),
  ('San Jose City Hall',
   ['4607',
    '',
    '8/29/2013 14:42',
    'San Jose City Hall',
    '10',
    '8/29/2013 14:43',
    'San Jose City

In [14]:
# Объединение по ключу
start_trips = station_idx.join(trips_start)
end_trips = station_idx.join(trips_end)

## 5. Создание модели для Trip & Station

In [15]:
from typing import NamedTuple
from datetime import datetime
from functools import reduce

In [16]:
def initStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str
    
    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )

def initTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str
        
    for trip in trips:
        try:
            yield Trip(                             
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
            ) 
        except:
            pass

In [17]:
# Применяем модель данных для Stations
stations_ = stations.mapPartitions(initStation)

In [18]:
stations_.first()

Station(station_id=2, name='San Jose Diridon Caltrain Station', lat=37.329732, long=-121.90178200000001, dockcount=27, landmark='San Jose', installation=datetime.datetime(2013, 8, 6, 0, 0))

In [19]:
# Применяем модель данных для Trip
trips_= trips.mapPartitions(initTrip)

In [20]:
trips_.first()

Trip(trip_id=4130, duration=71, start_date=datetime.datetime(2013, 8, 29, 10, 16), start_station_name='Mountain View City Hall', start_station_id=27, end_date=datetime.datetime(2013, 8, 29, 10, 17), end_station_name='Mountain View City Hall', end_station_id='27', bike_id=48, subscription_type='Subscriber', zip_code='97214')

In [21]:
trips_start = trips_.keyBy(lambda x: x.start_station_name)

In [22]:
# Расчет среденего времени для каждого парковочного места
avg = trips_start.mapValues(lambda x: x.duration).groupByKey()\
.mapValues(lambda y: reduce(lambda a, b: (a+b), y)/len(y))

In [23]:
avg.top(10, lambda x: x[1])

[('University and Emerson', 7090.239417989418),
 ('California Ave Caltrain Station', 4628.005847953216),
 ('Redwood City Public Library', 4579.234741784037),
 ('Park at Olive', 4438.1613333333335),
 ('San Jose Civic Center', 4208.016938519448),
 ('Rengstorff Avenue / California Street', 4174.082373782108),
 ('Redwood City Medical Center', 3959.491961414791),
 ('Palo Alto Caltrain Station', 3210.6489815253435),
 ('San Mateo County Center', 2716.7700348432054),
 ('Broadway at Main', 2481.2537313432836)]

In [24]:
# Найдем первую поездку для каждой из стоянок
first_trip = trips_start.reduceByKey(lambda x, y: x if x.start_date < y.start_date else y)

In [25]:
first_trip.collect()

[('South Van Ness at Market',
  Trip(trip_id=4074, duration=1131, start_date=datetime.datetime(2013, 8, 29, 9, 24), start_station_name='South Van Ness at Market', start_station_id=66, end_date=datetime.datetime(2013, 8, 29, 9, 43), end_station_name='San Francisco Caltrain 2 (330 Townsend)', end_station_id='69', bike_id=317, subscription_type='Subscriber', zip_code='94115')),
 ('Santa Clara at Almaden',
  Trip(trip_id=4500, duration=109, start_date=datetime.datetime(2013, 8, 29, 13, 25), start_station_name='Santa Clara at Almaden', start_station_id=4, end_date=datetime.datetime(2013, 8, 29, 13, 27), end_station_name='Adobe on Almaden', end_station_id='5', bike_id=679, subscription_type='Subscriber', zip_code='95112')),
 ('Clay at Battery',
  Trip(trip_id=4283, duration=1712, start_date=datetime.datetime(2013, 8, 29, 11, 55), start_station_name='Clay at Battery', start_station_id=41, end_date=datetime.datetime(2013, 8, 29, 12, 23), end_station_name='Harry Bridges Plaza (Ferry Building)',

## 6. Выполненная лабораторная работа

### I. Велосипед с максимальным пробегом

In [26]:
bike_max_mileage = trips_.keyBy(lambda x: x.bike_id)

In [27]:
bike_duration = bike_max_mileage.mapValues(lambda x: x.duration).reduceByKey(lambda x1, x2: x1 + x2)

In [28]:
bike_duration_top = bike_duration.top(1, key=lambda x: x[1])[0][0]

In [29]:
bike_duration_top

535

### II. Найти наибольшее расстояние между станциями

In [30]:
from math import *


def distance_between_stations(latitude1, longitude1, latitude2, longitude2):
  earth_radius = 6371.0088
  latitude1, longitude1 = radians(latitude1), radians(longitude1)
  latitude2, longitude2 = radians(latitude2), radians(longitude2)

  difference_latitude = latitude2 - latitude1
  difference_longitude = longitude2 - longitude1
  
  haversinus_latitutde = (sin(difference_latitude / 2)) ** 2
  haversinus_longitude = (sin(difference_longitude / 2)) ** 2
  
  return 2 * earth_radius * sqrt(haversinus_latitutde + cos(latitude1)\
                                 * cos(latitude2) * haversinus_longitude)



In [31]:
distance = stations_.cartesian(stations_)\
                         .filter(lambda x: x[0].station_id != x[1].station_id)\
                         .map(lambda x: [x[0], x[1], distance_between_stations(x[0].lat, x[0].long, x[1].lat, x[1].long)])\
                         .keyBy(lambda x: (x[0].name, x[1].name))\
                         .reduce(lambda x1, x2: x1 if x1[1] > x2[1] else x2)

In [32]:
print(f"Максимальное расстояние между станциями: {distance[0]}, {distance[1][2]}.")

Максимальное расстояние между станциями: ('Ryland Park', 'Mezes Park'), 34.317914350160784.


### III. Найти путь велосипеда с максимальным пробегом через станции

In [33]:
bike_path = trips_.filter(lambda x: x.bike_id == bike_duration_top)\
.sortBy(lambda x: x.start_date).map(lambda x: (x.start_station_name, x.end_station_name))

In [34]:
bike_path.first()

('Post at Kearney', 'San Francisco Caltrain (Townsend at 4th)')

### IV. Найти количество велосипедов в системе.

In [35]:
count_bikes = trips_.map(lambda x: x.bike_id).distinct().count()

In [36]:
count_bikes

700

## V. Найти пользователей потративших на поездки более 3 часов.

In [37]:
users = trips_.filter(lambda x: x.duration > (3 * 60 * 60))\
.map(lambda x: x.zip_code).filter(lambda x: x != "").distinct()

In [38]:
users.take(10)

['58553',
 '94301',
 '94039',
 '94133',
 '93726',
 '94123',
 '4517',
 '29200',
 '45322',
 '94080']